In [4]:
import argparse
import os
import multiprocessing
import pickle
import argparse
import os
import multiprocessing
import pickle


from datasets import load_dataset

In [5]:
ds = load_dataset("google-research-datasets/natural_questions", "default")
topics = ds["question"].text

Generating validation split: 100%|██████████| 7830/7830 [00:13<00:00, 598.29 examples/s]


KeyError: 'question'

# Query Generation

In [ ]:
# 1. Install / Upgrade vLLM if needed
# !pip install --upgrade vllm
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM, SamplingParams
import os
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
# 2. Specify the model name from the Hugging Face Hub
model_name = "meta-llama/Llama-3.1-70B-Instruct"

# 3. Initialize the LLM engine
llm = LLM(model=model_name, tensor_parallel_size=2)


/home/adanato/miniconda3/envs/rag_bench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 01-31 16:47:10 __init__.py:183] Automatically detected platform cuda.


2025-01-31 16:47:10,720	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-31 16:47:17 config.py:520] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 01-31 16:47:27 config.py:1328] Defaulting to use mp for distributed inference
WARNING 01-31 16:47:27 arg_utils.py:1107] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 01-31 16:47:27 config.py:1483] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 01-31 16:47:27 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='meta-llama/Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_le

In [ ]:
dataset = []
my_columns = ["Column_A", "Column_B", "Column_C", "Column_D", "Column_E", "Column_F", "Column_G"]

# Create an empty DataFrame with those columns
df = pd.DataFrame(columns=my_columns)
prompts = [
    "what can I do to make friends",
]
tokenizer = AutoTokenizer.from_pretrained(model_name)

chats = []

for prompt in prompts:

    chat = [
        {"role": "user", "content": prompt},
    ]
    chats.append(chat)

templated_chats = [tokenizer.apply_chat_template(chat, tokenize=False) for chat in chats]

# 5. Define sampling (generation) parameters
sampling_params = SamplingParams(
    temperature=0.7, 
    top_p=0.95, 
    max_tokens=1024
)

# 6. Generate outputs
outputs = llm.generate(prompts, sampling_params)

# 7. Print generated text
for i, output in enumerate(outputs):
    prompt = prompts[i]
    generated_text = output.outputs[0].text  # Take the first (or best) output
    print(f"Prompt: {prompt}")
    print(f"Completion: {generated_text}\n")


# Google search

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()  # Ensure chromedriver is in your PATH or specify its path

try:
    # Navigate to Google's homepage
    driver.get("https://www.google.com")

    # Optionally, wait for the page to load completely
    time.sleep(2)  # You can replace this with an explicit wait if preferred

    # Locate the search input box using its 'name' attribute
    search_box = driver.find_element(By.NAME, "q")

    # Type your search query into the search box
    query = "Selenium WebDriver"
    search_box.send_keys(query)

    # Submit the search form (this simulates pressing Enter)
    search_box.send_keys(Keys.RETURN)

    # Wait a few seconds for the search results to load
    time.sleep(5)

    # Optionally, print the title of the current page to verify the results page loaded
    print("Page title is:", driver.title)

finally:
    # Close the browser window
    driver.quit()


# Result verfication. Rule based